In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from textblob import TextBlob
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [23]:
lyrics_df = pd.read_csv('taylor_swift_lyrics.csv', encoding= "ISO-8859-1")
lyrics_df['lyric_textblob'] = lyrics_df['lyric'].apply(lambda x: TextBlob(x))
lyrics_df['polarity'] = lyrics_df['lyric_textblob'].apply(lambda x: x.sentiment.polarity)
lyrics_df['subjectivity'] = lyrics_df['lyric_textblob'].apply(lambda x: x.sentiment.subjectivity)
lyrics_df.head()

,artist,album,track_title,track_n,lyric,line,year,lyric_textblob,polarity,subjectivity
0,Taylor Swift,Taylor Swift,Tim McGraw,1,He said the way my blue eyes shined,1,2006,"(H, e, , s, a, i, d, , t, h, e, , w, a, y, ...",0.0,0.1
1,Taylor Swift,Taylor Swift,Tim McGraw,1,Put those Georgia stars to shame that night,2,2006,"(P, u, t, , t, h, o, s, e, , G, e, o, r, g, ...",0.0,0.0
2,Taylor Swift,Taylor Swift,Tim McGraw,1,"I said, ""That's a lie""",3,2006,"(I, , s, a, i, d, ,, , "", T, h, a, t, ', s, ...",0.0,0.0
3,Taylor Swift,Taylor Swift,Tim McGraw,1,Just a boy in a Chevy truck,4,2006,"(J, u, s, t, , a, , b, o, y, , i, n, , a, ...",0.0,0.0
4,Taylor Swift,Taylor Swift,Tim McGraw,1,That had a tendency of gettin' stuck,5,2006,"(T, h, a, t, , h, a, d, , a, , t, e, n, d, ...",0.0,0.0


In [16]:
song_df = lyrics_df.groupby(['album','track_title'])['lyric'].apply(" ".join).reset_index()
song_df['song_textblob'] = song_df['lyric'].apply(lambda x: TextBlob(x))
song_df['polarity'] = song_df['song_textblob'].apply(lambda x: x.sentiment.polarity)
song_df['subjectivity'] = song_df['song_textblob'].apply(lambda x: x.sentiment.subjectivity)
song_df.head()

,album,track_title,lyric,song_textblob,polarity,subjectivity
0,1989,All You Had to Do Was Stay,People like you always want back The love they...,"(P, e, o, p, l, e, , l, i, k, e, , y, o, u, ...",0.201206,0.403947
1,1989,Bad Blood,"'Cause baby, now we've got bad blood You know ...","(', C, a, u, s, e, , b, a, b, y, ,, , n, o, ...",-0.234846,0.630275
2,1989,Blank Space,"Nice to meet you, where you been? I could show...","(N, i, c, e, , t, o, , m, e, e, t, , y, o, ...",-0.003736,0.615789
3,1989,Clean,The drought was the very worst When the flower...,"(T, h, e, , d, r, o, u, g, h, t, , w, a, s, ...",0.051141,0.600347
4,1989,How You Get The Girl,"Uh, uh, uh Uh, uh, uh Uh-uh, uh-uh Stand there...","(U, h, ,, , u, h, ,, , u, h, , U, h, ,, , ...",-0.078125,0.450000


In [22]:
display(song_df.sort_values('polarity').head())
display(song_df.sort_values('polarity', ascending = False).head())

,album,track_title,lyric,song_textblob,polarity,subjectivity
9,1989,Shake It Off,I stay out too late Got nothing in my brain Th...,"(I, , s, t, a, y, , o, u, t, , t, o, o, , ...",-0.482914,0.869336
1,1989,Bad Blood,"'Cause baby, now we've got bad blood You know ...","(', C, a, u, s, e, , b, a, b, y, ,, , n, o, ...",-0.234846,0.630275
72,Taylor Swift,Picture To Burn,"State the obvious, I didn't get my perfect fan...","(S, t, a, t, e, , t, h, e, , o, b, v, i, o, ...",-0.199145,0.606838
37,Red,I Knew You Were Trouble,"Once upon a time, a few mistakes ago I was in ...","(O, n, c, e, , u, p, o, n, , a, , t, i, m, ...",-0.196356,0.328283
23,Fearless,Tell Me Why,I took a chance I took a shot And you might th...,"(I, , t, o, o, k, , a, , c, h, a, n, c, e, ...",-0.190594,0.533363


,album,track_title,lyric,song_textblob,polarity,subjectivity
65,Taylor Swift,A Perfectly Good Heart,Why would you wanna break a perfectly good hea...,"(W, h, y, , w, o, u, l, d, , y, o, u, , w, ...",0.523529,0.525490
74,Taylor Swift,Stay Beautiful,"Cory's eyes are like a jungle He smiles, it's ...","(C, o, r, y, ', s, , e, y, e, s, , a, r, e, ...",0.437351,0.672917
12,1989,Welcome to New York,"Walking through a crowd, the village is aglow ...","(W, a, l, k, i, n, g, , t, h, r, o, u, g, h, ...",0.426636,0.658263
21,Fearless,Hey Stephen,"Hey Stephen, I know looks can be deceiving But...","(H, e, y, , S, t, e, p, h, e, n, ,, , I, , ...",0.353889,0.717778
33,Red,Everything Has Changed (Ft. Ed Sheeran),All I knew this morning when I woke Is I know ...,"(A, l, l, , I, , k, n, e, w, , t, h, i, s, ...",0.339293,0.442964


In [28]:
# Why is Shake It Off most negative song? 
display(lyrics_df[lyrics_df['track_title'] == 'Shake It Off'].sort_values('polarity').head())
display(lyrics_df[lyrics_df['track_title'] == 'Shake It Off'].sort_values('polarity', ascending = False).head())

,artist,album,track_title,track_n,lyric,line,year,lyric_textblob,polarity,subjectivity
3268,Taylor Swift,1989,Shake It Off,6,"And the haters gonna hate, hate, hate, hate, hate",34,2014,"(A, n, d, , t, h, e, , h, a, t, e, r, s, , ...",-0.80000,0.900000
3248,Taylor Swift,1989,Shake It Off,6,"And the haters gonna hate, hate, hate, hate, hate",14,2014,"(A, n, d, , t, h, e, , h, a, t, e, r, s, , ...",-0.80000,0.900000
3288,Taylor Swift,1989,Shake It Off,6,"And the haters gonna hate, hate, hate, hate, h...",54,2014,"(A, n, d, , t, h, e, , h, a, t, e, r, s, , ...",-0.80000,0.900000
3281,Taylor Swift,1989,Shake It Off,6,"And dirty, dirty cheats of the world you could...",47,2014,"(A, n, d, , d, i, r, t, y, ,, , d, i, r, t, ...",-0.51746,0.686508
3252,Taylor Swift,1989,Shake It Off,6,"And the fakers gonna fake, fake, fake, fake, fake",18,2014,"(A, n, d, , t, h, e, , f, a, k, e, r, s, , ...",-0.50000,1.000000


,artist,album,track_title,track_n,lyric,line,year,lyric_textblob,polarity,subjectivity
3284,Taylor Swift,1989,Shake It Off,6,And to the fella over there with the hella goo...,50,2014,"(A, n, d, , t, o, , t, h, e, , f, e, l, l, ...",0.700000,0.600000
3259,Taylor Swift,1989,Shake It Off,6,I'm dancing on my own (dancing on my own),25,2014,"(I, ', m, , d, a, n, c, i, n, g, , o, n, , ...",0.600000,1.000000
3239,Taylor Swift,1989,Shake It Off,6,I go on too many dates,5,2014,"(I, , g, o, , o, n, , t, o, o, , m, a, n, ...",0.500000,0.500000
3282,Taylor Swift,1989,Shake It Off,6,My ex-man brought his new girlfriend,48,2014,"(M, y, , e, x, -, m, a, n, , b, r, o, u, g, ...",0.136364,0.454545
3271,Taylor Swift,1989,Shake It Off,6,"Heartbreakers gonna break, break, break, break...",37,2014,"(H, e, a, r, t, b, r, e, a, k, e, r, s, , g, ...",0.000000,0.000000


In [45]:
song_df.groupby(['album']).mean()

,polarity,subjectivity
album,,
1989,0.054124,0.512417
Fearless,0.095117,0.520810
Red,0.100281,0.481302
Speak Now,0.103686,0.467733
Taylor Swift,0.164292,0.534923
reputation,0.069012,0.514501


In [101]:
song_df['song_length'] = song_df['lyric'].apply(lambda x: len(x.split()))
song_df['song_length_unique'] = song_df['lyric'].apply(lambda x: len(set(x.split())))
song_df['uniqueness'] = song_df['song_length_unique'] / song_df['song_length']
song_df.sort_values('uniqueness').head()
display(song_df[['album','track_title','uniqueness']].sort_values('uniqueness').head())
display(song_df[['album','track_title','uniqueness']].sort_values('uniqueness', ascending = False).head())

,album,track_title,uniqueness
8,1989,Out of the Woods,0.197853
6,1989,I Wish You Would,0.223801
90,reputation,Look What You Made Me Do,0.235189
0,1989,All You Had to Do Was Stay,0.242826
9,1989,Shake It Off,0.248739


,album,track_title,uniqueness
39,Red,Sad Beautiful Tragic,0.546448
16,Fearless,Breathe (Ft. Colbie Caillat),0.538462
17,Fearless,Change,0.524164
24,Fearless,The Best Day,0.502841
41,Red,State of Grace,0.493506
